In [1]:
import os
import json 
import logging

logging.basicConfig(
    filename='log/app.log',            # Specify the log file name
    level=logging.DEBUG,           # Set the log level (DEBUG, INFO, WARNING, ERROR, CRITICAL)
    format='%(asctime)s - %(levelname)s - %(message)s'  # Set the log format
)

# Load the environment configuration JSON data
json_path = 'env_config.json'
with open(json_path, 'r') as file:
    env_config = json.load(file)

hf_home = env_config['HF_HOME']
# Set the HF_HOME environment variable
os.environ['HF_HOME'] = hf_home
# Set the access token to huggingface hub
access_token = env_config['access_token']
os.environ['HUGGINGFACE_HUB_TOKEN'] = access_token
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1,2,3"

from pytorch_lightning import Trainer

trainer = Trainer(accelerator="gpu", devices=2, strategy='ddp_notebook', max_epochs=10)  # strategy='ddp' or 'ddp_spawn'


/afs/crc.nd.edu/user/d/dpan/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/afs/crc.nd.edu/user/d/dpan/.local/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [2]:
import torch 
from transformers import ViTImageProcessor, ViTForImageClassification, ViTModel, ViTConfig
from maskgen.utils.vit_mod import ModViTForImageClassification
from PIL import Image
import requests
import matplotlib.pyplot as plt

import numpy as np
import cv2
from datasets import load_dataset,load_metric
from torchvision.transforms import (
    CenterCrop,
    Compose,
    Normalize,
    RandomHorizontalFlip,
    RandomResizedCrop,
    Resize,
    ToTensor,
)

# from accelerate import Accelerator


import logging
import sys

# accelerator = Accelerator()
# device = accelerator.device



In [3]:
# url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
# url = "http://farm3.staticflickr.com/2066/1798910782_5536af8767_z.jpg"
# url = "http://farm1.staticflickr.com/184/399924547_98e6cef97a_z.jpg"
url = "http://farm1.staticflickr.com/128/318959350_1a39aae18c_z.jpg"
image = Image.open(requests.get(url, stream=True).raw)

pretrained_name = 'google/vit-base-patch16-224'
# pretrained_name = 'vit-base-patch16-224-finetuned-imageneteval'
# pretrained_name = 'openai/clip-vit-base-patch32'
config = ViTConfig.from_pretrained(pretrained_name)
processor = ViTImageProcessor.from_pretrained(pretrained_name)
# get mean and std to unnormalize the processed images
mean, std = processor.image_mean, processor.image_std

original_model = ViTForImageClassification.from_pretrained(pretrained_name)
original_model.to('cpu')
state_dict = original_model.state_dict()
del original_model # remove the model to free up memory
# change to use_mask_token = True
pred_model = ModViTForImageClassification(config)
# load back the parameters from state_dict
# 为新的模型实例添加 mask_token 权重
if 'vit.embeddings.mask_token' not in state_dict:
    state_dict['vit.embeddings.mask_token'] = pred_model.vit.embeddings.mask_token

pred_model.load_state_dict(state_dict)

# set to eval mode
pred_model.eval()
print()

# 创建base model 

In [4]:
from peft import LoraConfig, get_peft_model
# We use the same foundation model
exp_base_model = ViTModel.from_pretrained(pretrained_name)
exp_base_model.to('cpu')

# convert to peft model and ready to use LoRA 
# 手动列出所有层的目标模块

target_modules = []
num_layers = 12  # BERT-base 有 12 层
for i in range(num_layers):
    target_modules.extend([
        f"encoder.layer.{i}.attention.attention.query",
        f"encoder.layer.{i}.attention.attention.key",
        f"encoder.layer.{i}.attention.attention.value",
        f"encoder.layer.{i}.attention.output.dense",
        f"encoder.layer.{i}.intermediate.dense",
        f"encoder.layer.{i}.output.dense"
    ])

lora_config = LoraConfig(
    r=4,  # 低秩矩阵的秩
    lora_alpha=32,  # LoRA 的缩放因子
    target_modules= target_modules,  # 目标模块
    lora_dropout=0.1  # Dropout 概率
)
exp_base_model = get_peft_model(exp_base_model, lora_config)

# exp_base_model = ViTModel.from_pretrained(pretrained_name)

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from maskgen.vision_models.vision_maskgen_clip import MaskGeneratingModel
mask_gen_model = MaskGeneratingModel(base_model=exp_base_model, pred_model=pred_model, hidden_size=config.hidden_size, num_classes=config.num_labels)
# if torch.cuda.device_count() > 1:
#     print("Let's use", torch.cuda.device_count(), "GPUs!")
#     # dim = 0 [30, xxx] -> [10, ...], [10, ...], [10, ...] on 3 GPUs
#     mask_gen_model = torch.nn.DataParallel(mask_gen_model)
print()

In [6]:
from torch.utils.data import DataLoader
# def load_data(seed=42): 
#     dataset = load_dataset("mrm8488/ImageNet1K-val")
#     dataset = dataset['train']
#     splits = dataset.train_test_split(test_size=0.1, seed=seed)
#     test_ds = splits['test']
#     splits = splits['train'].train_test_split(test_size=0.1, seed=seed)
#     train_ds = splits['train']
#     val_ds = splits['test']
#     return train_ds, val_ds, test_ds

# train_ds, val_ds, test_ds = load_data()
# dataset = load_dataset("mrm8488/ImageNet1K-val")['train']
dataset = load_dataset('imagenet-1k', split='train', streaming=False, token=access_token, trust_remote_code=True)

normalize = Normalize(mean=processor.image_mean, std=processor.image_std)
if "height" in processor.size:
    size = (processor.size["height"], processor.size["width"])
    crop_size = size
    max_size = None
elif "shortest_edge" in processor.size:
    size = processor.size["shortest_edge"]
    crop_size = (size, size)
    max_size = processor.size.get("longest_edge")

transforms = Compose(
        [
            RandomResizedCrop(crop_size),
            RandomHorizontalFlip(),
            ToTensor(),
            normalize,
        ]
    )

def preprocess(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

# train_ds.set_transform(preprocess)
# test_ds.set_transform(preprocess)
dataset.set_transform(preprocess)


batch_size = 1024
# train_dataloader = DataLoader(train_ds, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, num_workers=num_workers)
# train_dataloader = DataLoader(test_ds, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, num_workers=num_workers)
train_dataloader = DataLoader(dataset, batch_size=batch_size, collate_fn=collate_fn, shuffle=True, num_workers=31)

# num_steps = 5
# mini_batch_size = 256
# ppo_epochs = 2
# optimizer = torch.optim.Adam(mask_gen_model.parameters(), lr=1e-4, weight_decay=1e-5)
# optimizer = torch.optim.Adam(mask_gen_model.parameters(), lr=1e-4)


In [7]:
# 使用 Trainer 进行训练
trainer.fit(mask_gen_model, train_dataloader)

[W814 21:12:25.020007593 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [localhost]:47627 (errno: 97 - Address family not supported by protocol).
[W814 21:12:25.153222464 socket.cpp:697] [c10d] The client socket cannot be initialized to connect to [localhost]:47627 (errno: 97 - Address family not supported by protocol).


Epoch 0:   0%|          | 0/626 [00:00<?, ?it/s] 

/afs/crc.nd.edu/user/d/dpan/.local/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:900: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
/afs/crc.nd.edu/user/d/dpan/.local/lib/python3.12/site-packages/PIL/TiffImagePlugin.py:900: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
W0814 21:14:12.324000 22976325862464 torch/multiprocessing/spawn.py:146] Terminating process 2410798 via signal SIGTERM


ProcessRaisedException: 

-- Process 0 terminated with the following error:
Traceback (most recent call last):
  File "/afs/crc.nd.edu/user/d/dpan/.local/lib/python3.12/site-packages/torch/multiprocessing/spawn.py", line 76, in _wrap
    fn(i, *args)
  File "/afs/crc.nd.edu/user/d/dpan/.local/lib/python3.12/site-packages/pytorch_lightning/strategies/launchers/multiprocessing.py", line 173, in _wrapping_function
    results = function(*args, **kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/afs/crc.nd.edu/user/d/dpan/.local/lib/python3.12/site-packages/pytorch_lightning/trainer/trainer.py", line 574, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/afs/crc.nd.edu/user/d/dpan/.local/lib/python3.12/site-packages/pytorch_lightning/trainer/trainer.py", line 981, in _run
    results = self._run_stage()
              ^^^^^^^^^^^^^^^^^
  File "/afs/crc.nd.edu/user/d/dpan/.local/lib/python3.12/site-packages/pytorch_lightning/trainer/trainer.py", line 1025, in _run_stage


In [ ]:
from tqdm import tqdm
import os
from datetime import datetime

# Wrap the model with DataParallel
mask_gen_model = torch.nn.DataParallel(mask_gen_model)
pred_model = torch.nn.DataParallel(pred_model)

mask_gen_model = mask_gen_model.to(device)
pred_model = pred_model.to(device)

def save_model(model, model_prefix):
    state_dict = model.module.state_dict() if isinstance(model, torch.nn.DataParallel) else model.state_dict()
    model_name = model_prefix + datetime.now().strftime("%Y%m%d-%H%M%S") + ".pt"
    model_path = os.path.join("trained", model_name)
    torch.save(state_dict, model_path)

for epoch in range(5):
    pbar = tqdm(train_dataloader)
    for idx, data in enumerate(pbar):
        pixel_values = data['pixel_values'].to(device)
        # print("pixel_values", pixel_values.shape)

        loss_dict = train_one_batch(pred_model, pixel_values, optimizer, num_steps=num_steps, mini_batch_size=mini_batch_size, ppo_epochs=ppo_epochs)
        desc = f"Epoch {epoch+1}, Step {idx+1}: Loss = {loss_dict['loss']:.4f}, " \
               f"Actor Loss = {loss_dict['actor_loss']:.4f}, " \
               f"Critic Loss = {loss_dict['critic_loss']:.4f}, " \
               f"Entropy = {loss_dict['entropy']:.4f}, " \
               f"Returns = {loss_dict['returns']:.4f}"
        pbar.set_description(desc)
        if idx % 20 == 0:
            print(f"{desc}")
            # save the model
            save_model(mask_gen_model, "vision_clip_allclass_")
            
        


#  Step 1: Loss = 2.6805, Actor Loss = -3.1051, Critic Loss = 11.5714, Entropy = 0.6860, Returns = 2.9787

#Step 1: Loss = 0.5375, Actor Loss = -1.7771, Critic Loss = 4.6292, Entropy = 0.6860, Returns = 1.6055

: 

In [ ]:
from maskgen.utils.img_utils import plot_overlap_np
from maskgen.utils import idx_to_selector

# load mask_gen_model
# mask_gen_model.load_state_dict(torch.load("trained/vision_mask_gen_lora_clip_20240807-125254.pt"))
# mask_gen_model.load_state_dict(torch.load("trained/vision_clip_20240812-112600.pt"))




mask_gen_model.eval()


# url = 'http://images.cocodataset.org/val2017/000000039769.jpg'
# url = "http://farm3.staticflickr.com/2066/1798910782_5536af8767_z.jpg"
# url = "http://farm1.staticflickr.com/184/399924547_98e6cef97a_z.jpg"
url = "http://farm1.staticflickr.com/128/318959350_1a39aae18c_z.jpg"
image = Image.open(requests.get(url, stream=True).raw)
inputs = processor(images=image, return_tensors="pt")
# inputs.to(device)


# test_dataloader = DataLoader(dataset, batch_size=1, collate_fn=collate_fn, shuffle=True)
# inputs = next(iter(test_dataloader))

with torch.no_grad():

    img = inputs['pixel_values']
    img = img.to(device)
    pred_class = pred_model(img).logits.argmax(-1)
    predicted_class_idx = pred_class.item()

plt.title(f"Method: MaskGen Predicted class: {pred_model.config.id2label[predicted_class_idx]}")
size = 14
N = inputs['pixel_values'].shape[0]

# sim = mask_gen_model.forward(pixel_values=inputs['pixel_values'])
label = torch.tensor([predicted_class_idx]).to(device)
dist, value = mask_gen_model.get_dist_critic(img, label.unsqueeze(1))
sim_logits = dist.logits
# sim = torch.sigmoid(sim_logits)
# sim = mask_gen_model.get_attribution(inputs['pixel_values'])
# selector = idx_to_selector(pred_class, 1000).unsqueeze(1) # [N, 1, n_classes]
# sim = (sim * selector).sum(-1)
# sim = sim[:,:, predicted_class_idx]

heatmap = torch.sigmoid(sim_logits).reshape(N, size, size)

heatmap = heatmap.squeeze(0).detach().cpu().numpy()
img = img.squeeze(0).detach().cpu().numpy().transpose(1, 2, 0)

img_int, heatmap_img = plot_overlap_np(img, heatmap, mean, std)

: 

In [ ]:
print(heatmap.min())
print(heatmap.max())
print(heatmap.mean())

: 

In [ ]:
img.shape

: 

In [ ]:
# save model to trained folder
save_model(mask_gen_model, "vision_clip_final_")


: 

In [ ]:
# # how to check the size of the model in MB
# import os
# print(f"Model size: {os.path.getsize(model_path) / 1e6:.2f} MB")
device

: 

: 